In [1]:
from utils import load_my_data
from utils import extract_DenseSift_descriptors
from utils import build_codebook
from utils import input_vector_encoder
from classifier import svm_classifier
import spm

import numpy as np
import argparse

from sklearn.svm import SVC

import mnist

/datasets/home/98/998/cg260sax/.conda/envs/cogs/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/datasets/home/98/998/cg260sax/.conda/envs/cogs/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Train
X=mnist.train_images().astype('uint8')
y=mnist.train_labels().astype('uint8')

In [3]:
#Test
XTest=mnist.test_images().astype('uint8')[:7500]
yTest=mnist.test_labels().astype('uint8')[:7500]

In [4]:
##Delete this
XTrain=X[:20000]
yTrain=y[:20000]

In [ ]:
# allPreds=[]
# for VOC_SIZE in [50,100]:
#     for PYRAMID_LEVEL in [1,2]:
#         print "Codebook Size: {:d}".format(VOC_SIZE)
#         print "Pyramid level: {:d}".format(PYRAMID_LEVEL)


#         x_feature = [extract_DenseSift_descriptors(img) for img in XTrain]
#         print("Features extracted for all images")
#         x_kp, x_des = zip(*x_feature)
#         print "Building the codebook, it will take some time"
#         codebook = build_codebook(x_des, VOC_SIZE)#spm.VOC_SIZE
#         print "Spatial Pyramid Matching encoding"
#         X_train = [spm.spatial_pyramid_matching(XTrain[i],
#                                       x_des[i],
#                                       codebook,
#                                       level=PYRAMID_LEVEL)#spm.PYRAMID_LEVEL
#                                       for i in xrange(len(x_des))]
#         X_train =np.asarray(X_train)
# ###############################################################################################################
#         x_test_feature = [extract_DenseSift_descriptors(img) for img in XTest]
#         x_kp_test, x_des_test = zip(*x_test_feature)
#         print "Spatial Pyramid Matching encoding"
#         X_test = [spm.spatial_pyramid_matching(XTest[i],
#                                       x_des_test[i],
#                                       codebook,
#                                       level=PYRAMID_LEVEL)#spm.PYRAMID_LEVEL
#                                       for i in xrange(len(x_des_test))]
#         X_test =np.asarray(X_test)

In [5]:
VOC_SIZE = 200
PYRAMID_LEVEL = 2
print "Codebook Size: {:d}".format(VOC_SIZE)
print "Pyramid level: {:d}".format(PYRAMID_LEVEL)


x_feature = [extract_DenseSift_descriptors(img) for img in XTrain]
print("Features extracted for all images")
x_kp, x_des = zip(*x_feature)
print "Building the codebook, it will take some time"
codebook = build_codebook(x_des, VOC_SIZE)#spm.VOC_SIZE
print "Spatial Pyramid Matching encoding"
X_train = [spm.spatial_pyramid_matching(XTrain[i],
                              x_des[i],
                              codebook,
                              level=PYRAMID_LEVEL)#spm.PYRAMID_LEVEL
                              for i in xrange(len(x_des))]
X_train =np.asarray(X_train)
###############################################################################################################
x_test_feature = [extract_DenseSift_descriptors(img) for img in XTest]
x_kp_test, x_des_test = zip(*x_test_feature)
print "Spatial Pyramid Matching encoding"
X_test = [spm.spatial_pyramid_matching(XTest[i],
                              x_des_test[i],
                              codebook,
                              level=PYRAMID_LEVEL)#spm.PYRAMID_LEVEL
                              for i in xrange(len(x_des_test))]
X_test =np.asarray(X_test)

Codebook Size: 200
Pyramid level: 2
Features extracted for all images
Building the codebook, it will take some time
Spatial Pyramid Matching encoding
Spatial Pyramid Matching encoding


In [13]:
# x_test_feature = [extract_DenseSift_descriptors(img) for img in XTest]
# x_kp_test, x_des_test = zip(*x_test_feature)
# print "Spatial Pyramid Matching encoding"
# X_test = [spm.spatial_pyramid_matching(XTest[i],
#                               x_des_test[i],
#                               codebook,
#                               level=PYRAMID_LEVEL)#spm.PYRAMID_LEVEL
#                               for i in xrange(len(x_des_test))]
# X_test =np.asarray(X_test)

Spatial Pyramid Matching encoding


In [7]:
clf=SVC(C=10**4, kernel='rbf', degree=3,  coef0=0.0, shrinking=True, 
            probability=False, tol=0.001, cache_size=200, 
            class_weight=None, verbose=False, max_iter=-1, random_state=None)
clf.fit(X_train,yTrain)
preds=clf.predict(X_test)
#allPreds.append(preds)
accuracy=(sum([1 for i,j in zip(preds,yTest) if i==j])*1.0)/len(preds)
print(accuracy)

0.935333333333


In [ ]:
#% matplotlib inline
from matplotlib import pyplot as plt
import itertools
import numpy as np
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [22]:
from sklearn import metrics

cm=metrics.confusion_matrix(yTest,preds)

In [25]:
print('Confusion matrix for Vocab =200 and pyramid depth=2')
print(cm)

Confusion matrix for Vocab =200 and pyramid depth=2
[[703   1   1   0   1   0   3   1   8   1]
 [  0 836   1   4   0   1   4   0   3   0]
 [  5   0 750  11   1   2   3   6   5   1]
 [  0   1  14 712   0   3   0   8  10   7]
 [  0   0   2   1 693   0   6   0   2  45]
 [  1   0   0  37   2 616   1   0  15   4]
 [  7   1   1   0   6   2 685   0   4   0]
 [  2   4  11  13   1   0   0 681   9  41]
 [ 14   2   8  13  11  10   3  17 635  18]
 [  5   6   3  15  13   4   0  11   8 704]]


In [18]:
'VOC Size - 50, Pyramid level 1, Accuracy = 0.734666666667'
preds_test_1 = preds
from sklearn import metrics

cm = metrics.confusion_matrix(yTest, preds_test_1, range(10))

print (cm)
'''
[[634   2   2   0  12   3  28   2  25  11]
 [  1 829   0   5   2   0   4   2   6   0]
 [  9   3 492 171   5  30   6  49  17   2]
 [  0   8 117 489   0  11   0 103  16  11]
 [ 10   4   8   0 592  13  26   9  17  70]
 [  8   4  56  45  15 403  10  26  76  33]
 [ 33  10   2   0  29   3 613   0  14   2]
 [  3   6   2  66   3   8   0 592  23  59]
 [ 39  12  17  80  14  60  13  87 339  70]
 [ 30   7   3  12  52  67   2  23  46 527]]

'''

[[634   2   2   0  12   3  28   2  25  11]
 [  1 829   0   5   2   0   4   2   6   0]
 [  9   3 492 171   5  30   6  49  17   2]
 [  0   8 117 489   0  11   0 103  16  11]
 [ 10   4   8   0 592  13  26   9  17  70]
 [  8   4  56  45  15 403  10  26  76  33]
 [ 33  10   2   0  29   3 613   0  14   2]
 [  3   6   2  66   3   8   0 592  23  59]
 [ 39  12  17  80  14  60  13  87 339  70]
 [ 30   7   3  12  52  67   2  23  46 527]]
